In [1]:
import os
import sys
import pandas as pd
import geopandas as gpd
import numpy as np

In [2]:
gdf = gpd.read_file("shapefile/F120_beans_2023_rotated_qgis.shp")
gdf['fieldID'] = gdf['fieldID'].astype(int)
gdf = gdf.sort_values('fieldID')
gdf['fieldID'] = list(range(1, 457))

In [3]:
# def add_plot_number(gdf):
#     gdf = gdf.copy()
#     gdf['plot_number'] = np.nan
    
#     for index, row in gdf.iterrows():
#         fieldID = row['fieldID']
#         range_num = 20 - (fieldID - 1) // 24
#         col_num = (fieldID - 1) % 24 + 1
#         plot_number = f"{range_num:02}{col_num:02}"
#         gdf.at[index, 'plot_number'] = plot_number.zfill(4)
    
#     return gdf
def add_plot_number(gdf):
    gdf = gdf.copy()
    gdf['plot_number'] = np.nan
    
    for index, row in gdf.iterrows():
        fieldID = row['fieldID']
        range_num = 19 - (fieldID - 1) // 24
        col_num = (fieldID - 1) % 24 + 1
        plot_number = f"{range_num:02}{col_num:02}"
        gdf.at[index, 'plot_number'] = plot_number.zfill(4)
    
    return gdf

gdf = add_plot_number(gdf)

In [4]:
gdf['plot_number'] = gdf['plot_number'].astype(int).astype(str).str.zfill(4)

In [5]:
gdf = gdf.drop('fieldID', axis=1).rename(columns={'plot_number': 'fieldID'})

In [6]:
gdf.to_file("shapefile/F120_beans_2023_rotated_qgis_updated.shp")